In [ ]:
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
# Making sure you're using UTF-8 locale to install LangChain
import locale

locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 17.9 MB/s eta 0:00:00


In [ ]:
from google.colab import files

uploaded_files = files.upload()

Saving assistance.csv to assistance.csv


In [ ]:
# from langchain.document_loaders import GitHubIssuesLoader

# loader = GitHubIssuesLoader(repo="huggingface/peft", access_token=ACCESS_TOKEN, include_prs=False, state="all")

# docs = loader.load()

import pandas as pd

df = pd.read_csv("assistance.csv")

# Concatenate the title and content columns to create a single text column
df['full_text'] = df['title'] + "\n\n" + df['content']


print(df.head())


                                                 url  \
0  https://assistance.free.fr/articles/telephonie...   
1  https://assistance.free.fr/articles/telephone-...   
2  https://assistance.free.fr/articles/cout-du-pa...   
3  https://assistance.free.fr/articles/lenteur-de...   
4  https://assistance.free.fr/articles/disney-plu...   

                                               title  \
0  Téléphonie fixe : restreindre la présentation ...   
1  Téléphone fixe : tonalité inhabituelle au décr...   
2  Coût du passage de mon abonnement Freebox de l...   
3    Lenteur de navigation en Ethernet : que faire ?   
4                  Disney+ : résilier mon abonnement   

                                             content  \
0  Cette option vous permet de  dissimuler votre ...   
1  Lorsqu'un message vocal est en attente sur vot...   
2  L’ installation de la Fibre à votre domicile  ...   
3  Étape 1 Si vous êtes en  ADSL , assurez-vous q...   
4  Rendez-vous dans votre  Espace Abonné  et i

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)

# Convert all values in the 'full_text' column to strings
df['full_text'] = df['full_text'].astype(str)

# Preprocess the documents (e.g., split them into chunks)
chunked_docs = [splitter.split_text(doc) for doc in df['full_text']]

# Join the chunks within each document into a single string
texts = [' '.join(chunks) for chunks in chunked_docs]

# Define a simple class to represent documents
class Document:
    def __init__(self, content, metadata=None):
        self.page_content = content
        self.metadata = metadata if metadata is not None else {}

# Create a list of Document objects
documents = [Document(text, metadata={'id': i}) for i, text in enumerate(texts)]


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

db = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name="BAAI/bge-m3"))

# BAAI/bge-base-en-v1.5

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

In [ ]:
from langchain_core.runnables import RunnablePassthrough

retriever = db.as_retriever()

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain

In [ ]:
question = "Comment restreindre la présentation de mon numéro?"

In [ ]:
llm_chain.invoke({"context": "", "question": question})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n\n\n</s>\n<|user|>\nComment restreindre la présentation de mon numéro?\n</s>\n<|assistant|>\n\n  Pour restreindre la présentation de votre numéro, vous pouvez suivre les étapes suivantes :\n\n1. Accédez à votre compte téléphonique et recherchez l\'option "Privacy" ou "Confidentialité". Cela peut varier selon le fournisseur de services.\n\n2. Activez le mode "Restreint" ou "Privé" pour votre numéro. Cela signifie que seuls les appels d\'appelers réguliers seront acceptés, tandis que les appels anonymes, les appels internationales et les appels à tarifs spéciaux seront rejetés.\n\n3. Définissez les personnes autorisées à contacter votre numéro en créant une liste de contacts préférés ou en ajoutant des numéros à votre carnet d\'adresses. Les appels venant de ces numéros seront toujours acceptés, même si votre numéro est en mode "Restreint".\n\n4. Vous pouvez également configurer votre voicem

In [ ]:
rag_chain.invoke(question)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n[Document(page_content="Téléphonie fixe : restreindre la présentation de mon numéro Cette option vous permet de  dissimuler votre numéro  (masquer) à vos correspondants de manière permanente ou non.\\xa0 Décrochez votre combiné Freebox puis :\\xa0 Activer le secret d\'appel permanent :  *31# Désactiver le secret d\'appel permanent :  #31# Connaître le statut du service :  *#31# Activer le mode secret pour le prochain appel :  *31*  <  numéro  > Désactiver le mode secret pour le prochain appel :  #31*  <  numéro  > Sur certains modèles de téléphone, il faudra appuyer sur la touche  appel  après sur la touche  appel  après avoir saisi la combinaison de touches.\\xa0 Ce service est également paramétrable depuis votre  Espace Abonné , rubrique  Téléphonie ,  Gestion de mes services téléphonie  puis option  Restriction de présentation d\'identité . Après modification, veillez à cliquer sur le b